In [1]:
import numpy as np

## Code

In [2]:
import numpy as np
import re
from collections import defaultdict


class word2vec():
    def __init__ (self):
        self.n = settings['n']
        self.eta = settings['learning_rate']
        self.epochs = settings['epochs']
        self.window = settings['window_size']
        pass
    
    
    # GENERATE TRAINING DATA
    def generate_training_data(self, settings, corpus):

        # GENERATE WORD COUNTS
        word_counts = defaultdict(int)
        for row in corpus:
            for word in row:
                word_counts[word] += 1

        self.v_count = len(word_counts.keys())

        # GENERATE LOOKUP DICTIONARIES
        self.words_list = sorted(list(word_counts.keys()),reverse=False)
        self.word_index = dict((word, i) for i, word in enumerate(self.words_list))
        self.index_word = dict((i, word) for i, word in enumerate(self.words_list))

        training_data = []
        # CYCLE THROUGH EACH SENTENCE IN CORPUS
        for sentence in corpus:
            sent_len = len(sentence)

            # CYCLE THROUGH EACH WORD IN SENTENCE
            for i, word in enumerate(sentence):
                
                #w_target  = sentence[i]
                w_target = self.word2onehot(sentence[i])

                # CYCLE THROUGH CONTEXT WINDOW
                w_context = []
                for j in range(i-self.window, i+self.window+1):
                    if j!=i and j<=sent_len-1 and j>=0:
                        w_context.append(self.word2onehot(sentence[j]))
                training_data.append([w_target, w_context])
        return np.array(training_data)


    # SOFTMAX ACTIVATION FUNCTION
    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)


    # CONVERT WORD TO ONE HOT ENCODING
    def word2onehot(self, word):
        word_vec = [0 for i in range(0, self.v_count)]
        word_index = self.word_index[word]
        word_vec[word_index] = 1
        return word_vec


    # FORWARD PASS
    def forward_pass(self, x):
        h = np.dot(self.w1.T, x)
        u = np.dot(self.w2.T, h)
        y_c = self.softmax(u)
        return y_c, h, u
                

    # BACKPROPAGATION
    def backprop(self, e, h, x):
        dl_dw2 = np.outer(h, e)  
        dl_dw1 = np.outer(x, np.dot(self.w2, e.T))

        # UPDATE WEIGHTS
        self.w1 = self.w1 - (self.eta * dl_dw1)
        self.w2 = self.w2 - (self.eta * dl_dw2)
        pass


    # TRAIN W2V model
    def train(self, training_data):
        # INITIALIZE WEIGHT MATRICES
        self.w1 = np.random.uniform(-0.8, 0.8, (self.v_count, self.n))     # context matrix
        self.w2 = np.random.uniform(-0.8, 0.8, (self.n, self.v_count))     # embedding matrix
        
        # CYCLE THROUGH EACH EPOCH
        for i in range(0, self.epochs):

            self.loss = 0

            # CYCLE THROUGH EACH TRAINING SAMPLE
            for w_t, w_c in training_data:

                # FORWARD PASS
                y_pred, h, u = self.forward_pass(w_t)
                
                # CALCULATE ERROR
                EI = np.sum([np.subtract(y_pred, word) for word in w_c], axis=0)

                # BACKPROPAGATION
                self.backprop(EI, h, w_t)

                # CALCULATE LOSS
                self.loss += -np.sum([u[word.index(1)] for word in w_c]) + len(w_c) * np.log(np.sum(np.exp(u)))
                self.loss += -2*np.log(len(w_c)) -np.sum([u[word.index(1)] for word in w_c]) + (len(w_c) * np.log(np.sum(np.exp(u))))
                
            print('EPOCH:',i, 'LOSS:', self.loss)
        pass


    # input a word, returns a vector (if available)
    def word_vec(self, word):
        w_index = self.word_index[word]
        v_w = self.w1[w_index]
        return v_w


    # input a vector, returns nearest word(s)
    def vec_sim(self, vec, top_n):

        # CYCLE THROUGH VOCAB
        word_sim = {}
        for i in range(self.v_count):
            v_w2 = self.w1[i]
            theta_num = np.dot(vec, v_w2)
            theta_den = np.linalg.norm(vec) * np.linalg.norm(v_w2)
            theta = theta_num / theta_den

            word = self.index_word[i]
            word_sim[word] = theta

        words_sorted = sorted(word_sim.items(), key=lambda word, sim: sim, reverse=True)

        for word, sim in words_sorted[:top_n]:
            print(word, sim)
            
        pass

    # input word, returns top [n] most similar words
    def word_sim(self, word, top_n):
        
        w1_index = self.word_index[word]
        v_w1 = self.w1[w1_index]

        # CYCLE THROUGH VOCAB
        word_sim = {}
        for i in range(self.v_count):
            v_w2 = self.w1[i]
            theta_num = np.dot(v_w1, v_w2)
            theta_den = np.linalg.norm(v_w1) * np.linalg.norm(v_w2)
            theta = theta_num / theta_den

            word = self.index_word[i]
            word_sim[word] = theta

        words_sorted = sorted(word_sim.items(), key=lambda word, sim:sim, reverse=True)

        for word, sim in words_sorted[:top_n]:
            print(word, sim)
            
        pass



In [3]:
settings = {}
settings['n'] = 5                   # dimension of word embeddings
settings['window_size'] = 2         # context window +/- center word
settings['min_count'] = 0           # minimum word count
settings['epochs'] = 5000           # number of training epochs
settings['neg_samp'] = 10           # number of negative words to use during training
settings['learning_rate'] = 0.01    # learning rate
np.random.seed(0)                   # set the seed for reproducibility

corpus = [['the','quick','brown','fox','jumped','over','the','lazy','dog']]

# INITIALIZE W2V MODEL
w2v = word2vec()

# generate training data
training_data = w2v.generate_training_data(settings, corpus)

# train word2vec model
w2v.train(training_data)

EPOCH: 0 LOSS: 115.711946046
EPOCH: 1 LOSS: 114.736293974
EPOCH: 2 LOSS: 113.832568632
EPOCH: 3 LOSS: 112.992428501
EPOCH: 4 LOSS: 112.208726729
EPOCH: 5 LOSS: 111.47530631
EPOCH: 6 LOSS: 110.786835298
EPOCH: 7 LOSS: 110.138673405
EPOCH: 8 LOSS: 109.526763334
EPOCH: 9 LOSS: 108.947541748
EPOCH: 10 LOSS: 108.397865917
EPOCH: 11 LOSS: 107.874952963
EPOCH: 12 LOSS: 107.376329274
EPOCH: 13 LOSS: 106.899788188
EPOCH: 14 LOSS: 106.443354426
EPOCH: 15 LOSS: 106.005254059
EPOCH: 16 LOSS: 105.583889059
EPOCH: 17 LOSS: 105.177815627
EPOCH: 18 LOSS: 104.785725681
EPOCH: 19 LOSS: 104.406430981
EPOCH: 20 LOSS: 104.038849466
EPOCH: 21 LOSS: 103.681993468
EPOCH: 22 LOSS: 103.334959495
EPOCH: 23 LOSS: 102.99691937
EPOCH: 24 LOSS: 102.667112505
EPOCH: 25 LOSS: 102.344839165
EPOCH: 26 LOSS: 102.029454568
EPOCH: 27 LOSS: 101.720363708
EPOCH: 28 LOSS: 101.417016813
EPOCH: 29 LOSS: 101.118905338
EPOCH: 30 LOSS: 100.825558431
EPOCH: 31 LOSS: 100.536539809
EPOCH: 32 LOSS: 100.251444996
EPOCH: 33 LOSS: 99.969

EPOCH: 311 LOSS: 63.8505446207
EPOCH: 312 LOSS: 63.8335176993
EPOCH: 313 LOSS: 63.8166797859
EPOCH: 314 LOSS: 63.8000279953
EPOCH: 315 LOSS: 63.7835594981
EPOCH: 316 LOSS: 63.7672715202
EPOCH: 317 LOSS: 63.7511613406
EPOCH: 318 LOSS: 63.7352262908
EPOCH: 319 LOSS: 63.7194637533
EPOCH: 320 LOSS: 63.7038711602
EPOCH: 321 LOSS: 63.6884459925
EPOCH: 322 LOSS: 63.6731857784
EPOCH: 323 LOSS: 63.6580880927
EPOCH: 324 LOSS: 63.6431505554
EPOCH: 325 LOSS: 63.6283708305
EPOCH: 326 LOSS: 63.6137466255
EPOCH: 327 LOSS: 63.5992756899
EPOCH: 328 LOSS: 63.5849558143
EPOCH: 329 LOSS: 63.5707848299
EPOCH: 330 LOSS: 63.5567606069
EPOCH: 331 LOSS: 63.5428810539
EPOCH: 332 LOSS: 63.5291441172
EPOCH: 333 LOSS: 63.5155477798
EPOCH: 334 LOSS: 63.5020900604
EPOCH: 335 LOSS: 63.4887690129
EPOCH: 336 LOSS: 63.4755827253
EPOCH: 337 LOSS: 63.4625293191
EPOCH: 338 LOSS: 63.4496069488
EPOCH: 339 LOSS: 63.4368138007
EPOCH: 340 LOSS: 63.4241480923
EPOCH: 341 LOSS: 63.4116080719
EPOCH: 342 LOSS: 63.3991920178
EPOCH: 3

EPOCH: 610 LOSS: 61.9792012379
EPOCH: 611 LOSS: 61.9767697312
EPOCH: 612 LOSS: 61.9743467467
EPOCH: 613 LOSS: 61.9719322401
EPOCH: 614 LOSS: 61.9695261679
EPOCH: 615 LOSS: 61.9671284869
EPOCH: 616 LOSS: 61.9647391548
EPOCH: 617 LOSS: 61.9623581295
EPOCH: 618 LOSS: 61.9599853698
EPOCH: 619 LOSS: 61.957620835
EPOCH: 620 LOSS: 61.9552644848
EPOCH: 621 LOSS: 61.9529162796
EPOCH: 622 LOSS: 61.9505761802
EPOCH: 623 LOSS: 61.9482441481
EPOCH: 624 LOSS: 61.9459201452
EPOCH: 625 LOSS: 61.943604134
EPOCH: 626 LOSS: 61.9412960774
EPOCH: 627 LOSS: 61.9389959388
EPOCH: 628 LOSS: 61.9367036823
EPOCH: 629 LOSS: 61.9344192722
EPOCH: 630 LOSS: 61.9321426735
EPOCH: 631 LOSS: 61.9298738514
EPOCH: 632 LOSS: 61.927612772
EPOCH: 633 LOSS: 61.9253594014
EPOCH: 634 LOSS: 61.9231137064
EPOCH: 635 LOSS: 61.9208756542
EPOCH: 636 LOSS: 61.9186452123
EPOCH: 637 LOSS: 61.9164223489
EPOCH: 638 LOSS: 61.9142070324
EPOCH: 639 LOSS: 61.9119992316
EPOCH: 640 LOSS: 61.9097989158
EPOCH: 641 LOSS: 61.9076060548
EPOCH: 642 

EPOCH: 884 LOSS: 61.5492521088
EPOCH: 885 LOSS: 61.5483406853
EPOCH: 886 LOSS: 61.5474326681
EPOCH: 887 LOSS: 61.5465280428
EPOCH: 888 LOSS: 61.5456267955
EPOCH: 889 LOSS: 61.5447289119
EPOCH: 890 LOSS: 61.5438343781
EPOCH: 891 LOSS: 61.5429431798
EPOCH: 892 LOSS: 61.5420553032
EPOCH: 893 LOSS: 61.5411707343
EPOCH: 894 LOSS: 61.540289459
EPOCH: 895 LOSS: 61.5394114636
EPOCH: 896 LOSS: 61.538536734
EPOCH: 897 LOSS: 61.5376652564
EPOCH: 898 LOSS: 61.5367970171
EPOCH: 899 LOSS: 61.5359320023
EPOCH: 900 LOSS: 61.5350701981
EPOCH: 901 LOSS: 61.534211591
EPOCH: 902 LOSS: 61.5333561671
EPOCH: 903 LOSS: 61.532503913
EPOCH: 904 LOSS: 61.5316548149
EPOCH: 905 LOSS: 61.5308088594
EPOCH: 906 LOSS: 61.5299660328
EPOCH: 907 LOSS: 61.5291263217
EPOCH: 908 LOSS: 61.5282897127
EPOCH: 909 LOSS: 61.5274561923
EPOCH: 910 LOSS: 61.5266257472
EPOCH: 911 LOSS: 61.525798364
EPOCH: 912 LOSS: 61.5249740295
EPOCH: 913 LOSS: 61.5241527303
EPOCH: 914 LOSS: 61.5233344532
EPOCH: 915 LOSS: 61.5225191851
EPOCH: 916 LO

EPOCH: 1178 LOSS: 61.3809074598
EPOCH: 1179 LOSS: 61.3805562012
EPOCH: 1180 LOSS: 61.380205876
EPOCH: 1181 LOSS: 61.3798564804
EPOCH: 1182 LOSS: 61.3795080107
EPOCH: 1183 LOSS: 61.379160463
EPOCH: 1184 LOSS: 61.3788138338
EPOCH: 1185 LOSS: 61.3784681192
EPOCH: 1186 LOSS: 61.3781233157
EPOCH: 1187 LOSS: 61.3777794194
EPOCH: 1188 LOSS: 61.3774364269
EPOCH: 1189 LOSS: 61.3770943344
EPOCH: 1190 LOSS: 61.3767531383
EPOCH: 1191 LOSS: 61.376412835
EPOCH: 1192 LOSS: 61.3760734211
EPOCH: 1193 LOSS: 61.3757348928
EPOCH: 1194 LOSS: 61.3753972467
EPOCH: 1195 LOSS: 61.3750604792
EPOCH: 1196 LOSS: 61.3747245868
EPOCH: 1197 LOSS: 61.3743895661
EPOCH: 1198 LOSS: 61.3740554136
EPOCH: 1199 LOSS: 61.3737221257
EPOCH: 1200 LOSS: 61.3733896991
EPOCH: 1201 LOSS: 61.3730581304
EPOCH: 1202 LOSS: 61.3727274161
EPOCH: 1203 LOSS: 61.3723975529
EPOCH: 1204 LOSS: 61.3720685374
EPOCH: 1205 LOSS: 61.3717403662
EPOCH: 1206 LOSS: 61.371413036
EPOCH: 1207 LOSS: 61.3710865434
EPOCH: 1208 LOSS: 61.3707608852
EPOCH: 1209 

...

EPOCH: 3759 LOSS: 61.1399854282
EPOCH: 3760 LOSS: 61.1399470469
EPOCH: 3761 LOSS: 61.1399086767
EPOCH: 3762 LOSS: 61.1398703173
EPOCH: 3763 LOSS: 61.1398319689
EPOCH: 3764 LOSS: 61.1397936314
EPOCH: 3765 LOSS: 61.1397553048
EPOCH: 3766 LOSS: 61.1397169891
EPOCH: 3767 LOSS: 61.1396786844
EPOCH: 3768 LOSS: 61.1396403905
EPOCH: 3769 LOSS: 61.1396021075
EPOCH: 3770 LOSS: 61.1395638354
EPOCH: 3771 LOSS: 61.1395255741
EPOCH: 3772 LOSS: 61.1394873238
EPOCH: 3773 LOSS: 61.1394490843
EPOCH: 3774 LOSS: 61.1394108556
EPOCH: 3775 LOSS: 61.1393726378
EPOCH: 3776 LOSS: 61.1393344308
EPOCH: 3777 LOSS: 61.1392962347
EPOCH: 3778 LOSS: 61.1392580494
EPOCH: 3779 LOSS: 61.1392198749
EPOCH: 3780 LOSS: 61.1391817112
EPOCH: 3781 LOSS: 61.1391435583
EPOCH: 3782 LOSS: 61.1391054162
EPOCH: 3783 LOSS: 61.1390672849
EPOCH: 3784 LOSS: 61.1390291644
EPOCH: 3785 LOSS: 61.1389910547
EPOCH: 3786 LOSS: 61.1389529558
EPOCH: 3787 LOSS: 61.1389148676
EPOCH: 3788 LOSS: 61.1388767902
EPOCH: 3789 LOSS: 61.1388387235
EPOCH: 3

EPOCH: 4032 LOSS: 61.1298921045
EPOCH: 4033 LOSS: 61.1298564783
EPOCH: 4034 LOSS: 61.1298208613
EPOCH: 4035 LOSS: 61.1297852537
EPOCH: 4036 LOSS: 61.1297496554
EPOCH: 4037 LOSS: 61.1297140663
EPOCH: 4038 LOSS: 61.1296784866
EPOCH: 4039 LOSS: 61.1296429162
EPOCH: 4040 LOSS: 61.129607355
EPOCH: 4041 LOSS: 61.1295718032
EPOCH: 4042 LOSS: 61.1295362606
EPOCH: 4043 LOSS: 61.1295007273
EPOCH: 4044 LOSS: 61.1294652032
EPOCH: 4045 LOSS: 61.1294296884
EPOCH: 4046 LOSS: 61.1293941829
EPOCH: 4047 LOSS: 61.1293586866
EPOCH: 4048 LOSS: 61.1293231996
EPOCH: 4049 LOSS: 61.1292877218
EPOCH: 4050 LOSS: 61.1292522533
EPOCH: 4051 LOSS: 61.1292167939
EPOCH: 4052 LOSS: 61.1291813438
EPOCH: 4053 LOSS: 61.129145903
EPOCH: 4054 LOSS: 61.1291104713
EPOCH: 4055 LOSS: 61.1290750489
EPOCH: 4056 LOSS: 61.1290396356
EPOCH: 4057 LOSS: 61.1290042316
EPOCH: 4058 LOSS: 61.1289688367
EPOCH: 4059 LOSS: 61.1289334511
EPOCH: 4060 LOSS: 61.1288980746
EPOCH: 4061 LOSS: 61.1288627074
EPOCH: 4062 LOSS: 61.1288273493
EPOCH: 406

EPOCH: 4305 LOSS: 61.1204961066
EPOCH: 4306 LOSS: 61.1204628511
EPOCH: 4307 LOSS: 61.1204296038
EPOCH: 4308 LOSS: 61.1203963645
EPOCH: 4309 LOSS: 61.1203631333
EPOCH: 4310 LOSS: 61.1203299102
EPOCH: 4311 LOSS: 61.1202966952
EPOCH: 4312 LOSS: 61.1202634883
EPOCH: 4313 LOSS: 61.1202302894
EPOCH: 4314 LOSS: 61.1201970986
EPOCH: 4315 LOSS: 61.1201639159
EPOCH: 4316 LOSS: 61.1201307413
EPOCH: 4317 LOSS: 61.1200975747
EPOCH: 4318 LOSS: 61.1200644162
EPOCH: 4319 LOSS: 61.1200312658
EPOCH: 4320 LOSS: 61.1199981233
EPOCH: 4321 LOSS: 61.119964989
EPOCH: 4322 LOSS: 61.1199318627
EPOCH: 4323 LOSS: 61.1198987444
EPOCH: 4324 LOSS: 61.1198656341
EPOCH: 4325 LOSS: 61.1198325319
EPOCH: 4326 LOSS: 61.1197994377
EPOCH: 4327 LOSS: 61.1197663516
EPOCH: 4328 LOSS: 61.1197332734
EPOCH: 4329 LOSS: 61.1197002033
EPOCH: 4330 LOSS: 61.1196671412
EPOCH: 4331 LOSS: 61.1196340871
EPOCH: 4332 LOSS: 61.119601041
EPOCH: 4333 LOSS: 61.1195680029
EPOCH: 4334 LOSS: 61.1195349728
EPOCH: 4335 LOSS: 61.1195019507
EPOCH: 433

EPOCH: 4582 LOSS: 61.1115811957
EPOCH: 4583 LOSS: 61.1115500471
EPOCH: 4584 LOSS: 61.1115189057
EPOCH: 4585 LOSS: 61.1114877714
EPOCH: 4586 LOSS: 61.1114566443
EPOCH: 4587 LOSS: 61.1114255243
EPOCH: 4588 LOSS: 61.1113944115
EPOCH: 4589 LOSS: 61.1113633058
EPOCH: 4590 LOSS: 61.1113322072
EPOCH: 4591 LOSS: 61.1113011157
EPOCH: 4592 LOSS: 61.1112700314
EPOCH: 4593 LOSS: 61.1112389541
EPOCH: 4594 LOSS: 61.111207884
EPOCH: 4595 LOSS: 61.111176821
EPOCH: 4596 LOSS: 61.1111457651
EPOCH: 4597 LOSS: 61.1111147164
EPOCH: 4598 LOSS: 61.1110836747
EPOCH: 4599 LOSS: 61.1110526401
EPOCH: 4600 LOSS: 61.1110216126
EPOCH: 4601 LOSS: 61.1109905922
EPOCH: 4602 LOSS: 61.110959579
EPOCH: 4603 LOSS: 61.1109285727
EPOCH: 4604 LOSS: 61.1108975736
EPOCH: 4605 LOSS: 61.1108665816
EPOCH: 4606 LOSS: 61.1108355966
EPOCH: 4607 LOSS: 61.1108046188
EPOCH: 4608 LOSS: 61.1107736479
EPOCH: 4609 LOSS: 61.1107426842
EPOCH: 4610 LOSS: 61.1107117275
EPOCH: 4611 LOSS: 61.1106807779
EPOCH: 4612 LOSS: 61.1106498354
EPOCH: 4613

EPOCH: 4873 LOSS: 61.1028067653
EPOCH: 4874 LOSS: 61.1027775767
EPOCH: 4875 LOSS: 61.1027483945
EPOCH: 4876 LOSS: 61.1027192186
EPOCH: 4877 LOSS: 61.102690049
EPOCH: 4878 LOSS: 61.1026608858
EPOCH: 4879 LOSS: 61.1026317289
EPOCH: 4880 LOSS: 61.1026025784
EPOCH: 4881 LOSS: 61.1025734342
EPOCH: 4882 LOSS: 61.1025442963
EPOCH: 4883 LOSS: 61.1025151648
EPOCH: 4884 LOSS: 61.1024860396
EPOCH: 4885 LOSS: 61.1024569207
EPOCH: 4886 LOSS: 61.1024278081
EPOCH: 4887 LOSS: 61.1023987019
EPOCH: 4888 LOSS: 61.102369602
EPOCH: 4889 LOSS: 61.1023405084
EPOCH: 4890 LOSS: 61.1023114211
EPOCH: 4891 LOSS: 61.1022823402
EPOCH: 4892 LOSS: 61.1022532655
EPOCH: 4893 LOSS: 61.1022241971
EPOCH: 4894 LOSS: 61.1021951351
EPOCH: 4895 LOSS: 61.1021660793
EPOCH: 4896 LOSS: 61.1021370299
EPOCH: 4897 LOSS: 61.1021079868
EPOCH: 4898 LOSS: 61.1020789499
EPOCH: 4899 LOSS: 61.1020499193
EPOCH: 4900 LOSS: 61.1020208951
EPOCH: 4901 LOSS: 61.1019918771
EPOCH: 4902 LOSS: 61.1019628654
EPOCH: 4903 LOSS: 61.10193386
EPOCH: 4904 